In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re
from scipy import stats
np.random.seed(12345678)

In [ ]:
data = pd.read_csv('gene_matrix_CNV.csv.gz', compression='gzip')

In [ ]:
data_nan = data.dropna()

In [ ]:
data_nan.to_csv('gene_matrix_CNV_noNA.csv.gz', index=False, compression='gzip')

# Differentialy expressed Genes with Copy Number

In [ ]:
data = pd.read_csv('gene_matrix_CNV_noNA.csv.gz', compression='gzip')

In [ ]:
data.shape

In [ ]:
import os

os.mkdir('p-values-matrix')
#os.mkdir('train_degs2')

In [ ]:
path = 'Splits_10CV/'
for split in range(0,10):
    print('{}/{}'.format(split,9))
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 
        
    train_final = []
    for i in range(len(list(data.columns))):
        resu = re.match('|'.join(train_cids),list(data.columns)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for i in range(len(list(data.columns))):
        resu = re.match('|'.join(val_cids),list(data.columns)[i])
        if resu:
            if resu.group(0) != '':
                val_final.append(i)

    train_final.insert(0, 1)
    val_final.insert(0, 1)
    df_train = data.iloc[:,train_final]
    df_val = data.iloc[:,val_final]
    labels_train = df_train.iloc[-1,:].values
    no_BDN = np.where(labels_train != 'Blood Derived Normal')[0]
    no_BDN = no_BDN.tolist()
    df_train = df_train.iloc[:,no_BDN]
    labels_train = labels_train[no_BDN]
    #labels_train = np.where(labels_train == 'Blood Derived Normal', 'Healthy', labels_train)
    labels_train = np.where(labels_train == 'Solid Tissue Normal', 'Healthy', labels_train)

    hlt_idx = np.where(labels_train == 'Healthy')[0]
    luad_idx = np.where(labels_train == 'LUAD')[0]
    lusc_idx = np.where(labels_train == 'LUSC')[0]
    
    # p-values and means
    p_value_hltluad = []
    p_value_hltlusc = []
    p_value_luscluad = []
    mean_dif_hltluad = []
    mean_dif_hltlusc = []
    mean_dif_luscluad = []
    for _, row in tqdm(df_train.iterrows()):
        if row[0] == 'X':
            continue
        comp_counter = 0
        
        hlt = row[hlt_idx].to_numpy(dtype='float')
        luad = row[luad_idx].to_numpy(dtype='float')
        lusc = row[lusc_idx].to_numpy(dtype='float')

        stats_hltluad = stats.ttest_ind(hlt,luad,equal_var=False)
        stats_hltlusc = stats.ttest_ind(hlt,lusc,equal_var=False)
        stats_luscluad = stats.ttest_ind(lusc,luad,equal_var=False)

        mean_dif_hltluad.append(np.abs(np.mean(hlt) - np.mean(luad)))
        mean_dif_hltlusc.append(np.abs(np.mean(hlt) - np.mean(lusc)))
        mean_dif_luscluad.append(np.abs(np.mean(lusc) - np.mean(luad)))

        p_value_hltluad.append(stats_hltluad.pvalue)
        p_value_hltlusc.append(stats_hltlusc.pvalue)
        p_value_luscluad.append(stats_luscluad.pvalue)
    
    p_values_df = pd.DataFrame()
    p_values_df['gene_name'] = df_train['gene_name'].values[:-1]
    p_values_df['HltLusc'] = p_value_hltlusc
    p_values_df['HltLuad'] = p_value_hltluad
    p_values_df['LuscLuad'] = p_value_luscluad
    p_values_df['MeanHltLuad'] = mean_dif_hltluad
    p_values_df['MeanHltLusc'] = mean_dif_hltlusc
    p_values_df['MeanLuscLuad'] = mean_dif_luscluad

    p_values_df.to_csv('p-values-matrix/p_values_train'+str(split)+'_noBDN.csv', index=False)
    decpgs = []
    for _, row in tqdm(p_values_df.iterrows()):
        comp_counter = 0
        if row['HltLuad'] <= 0.001/3 and row['MeanHltLuad'] >= 0.1:
                comp_counter += 1
        if row['HltLusc'] <= 0.001/3 and row['MeanHltLusc'] >= 0.1:
            comp_counter += 1
        if row['LuscLuad'] <= 0.001/3 and row['MeanLuscLuad'] >= 0.1:
            comp_counter += 1

        if comp_counter == 3:
            decpgs.append(row[0])
    
    decpgs.append('X')
    
    train_degs = df_train.loc[df_train['gene_name'].isin(decpgs)]
    train_degs = train_degs.replace('Solid Tissue Normal', 'Healthy')
    train_degs = train_degs.replace('X', 'Label')
    train_degs.to_csv('train_degs2/DEGs_CNV_train'+str(split)+'_p0-001_cov3_noBDN.csv', index=False)

In [ ]:
from glob import glob
from tqdm import tqdm
for file in tqdm(sorted(glob('train_degs/*cov3.csv'))):
    train_data = pd.read_csv(file)
    train_data = train_data.replace('Blood Derived Normal', 'Healthy')
    train_data = train_data.replace('Solid Tissue Normal', 'Healthy')
    train_data = train_data.replace('X', 'Label')
    name = file.split('/')[-1]
    train_data.to_csv('train_degs2/'+name, index=False)

In [ ]:
os.mkdir('train_degs2')

## Machine Learning with first split

In [ ]:
data_all = pd.read_csv('gene_matrix_CNV_noNA.csv.gz', compression='gzip')

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, f1_score
import xgboost as xgb
classes = np.array(['healthy', 'adeno', 'squa'])

ohe = preprocessing.OneHotEncoder(sparse=False)
ohe.fit(classes.reshape(-1,1))

In [ ]:
import os
os.mkdir('results_excels')

In [ ]:
ohe.transform(np.array(['adeno']).reshape(-1,1))

In [ ]:
def get_val_set(x, y, classes, percentage = 0.1):
    np.random.seed(42)  
    x_train = np.array([]).reshape(0,x.shape[1])
    y_train = np.array([]).reshape(0,y.shape[1])
    x_val = np.array([]).reshape(0,x.shape[1])
    y_val = np.array([]).reshape(0,y.shape[1])
    for c in classes:
        indexes = np.where(y.argmax(axis=1) == c)[0]
        np.random.shuffle(indexes)
        len_val = int(percentage * len(indexes))
        len_train = len(indexes) - len_val
        index_train = indexes[0:len_train]
        index_val = indexes[len_train:]
        x_train = np.concatenate([x_train, x[index_train,...]], axis=0)
        y_train = np.concatenate([y_train, y[index_train]], axis=0)
        x_val = np.concatenate([x_val, x[index_val,...]], axis=0)
        y_val = np.concatenate([y_val, y[index_val]], axis=0)
    
    index_train = list(range(x_train.shape[0]))
    index_val = list(range(x_val.shape[0]))
    np.random.shuffle(index_train)
    np.random.shuffle(index_val)
    
    return x_train[index_train,...],y_train[index_train], x_val[index_val,...], y_val[index_val]

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
path = 'Splits_10CV/'
train_accs = []
test_accs = []
train_F1 = []
test_F1 = []

n_genes = 12
writer_test = pd.ExcelWriter('results_excels/CNV_'+str(n_genes)+'gen_XGBOOST_test.xlsx', engine='openpyxl') 
writer_train = pd.ExcelWriter('results_excels/CNV_'+str(n_genes)+'gen_XGBOOST_train.xlsx', engine='openpyxl') 
all_labels = data_all.iloc[-1,:]
for split in tqdm(range(10)):
    print(10*'-')
    print('Split {}/{}'.format(split,10))
    print(10*'-')
    
    print('Data read...')
    data = pd.read_csv('mrmrDEGs/mrmrDEGs_LC_CNV_3classes_p0-001_m0-1_cov3_split'+str(split)+'.csv')
    
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 

    train_final = []
    for i in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(train_cids),list(data['Case_IDs'].values)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for j in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(val_cids),list(data['Case_IDs'].values)[j])
        if resu:
            if resu.group(0) != '':
                val_final.append(j)

    #train_final.insert(0, 1)
    #val_final.insert(0, 1)
    df_train = data.iloc[train_final,]
    df_val = data.iloc[val_final,]

    case_ids_val = df_val['Case_IDs']
    #val_df_all = data_all[case_ids_val]
    y_val = all_labels[case_ids_val].values
    y_val = np.where(y_val == 'Blood Derived Normal', 'healthy', y_val)
    y_val = np.where(y_val == 'Solid Tissue Normal', 'healthy', y_val)
    y_val = np.where(y_val == 'LUAD', 'adeno', y_val)
    y_val = np.where(y_val == 'LUSC', 'squa', y_val)
    
    case_ids_train = df_train['Case_IDs']
    #train_df_all = data_all[case_ids_train]
    y_train = all_labels[case_ids_train].values
    y_train = np.where(y_train == 'Blood Derived Normal', 'healthy', y_train)
    y_train = np.where(y_train == 'Solid Tissue Normal', 'healthy', y_train)
    y_train = np.where(y_train == 'LUAD', 'adeno', y_train)
    y_train = np.where(y_train == 'LUSC', 'squa', y_train)
    
    x_train = df_train.iloc[:,1:n_genes+1].values
    x_val = df_val.iloc[:,1:n_genes+1].values
    y_train_ohe = ohe.transform(y_train.reshape(-1,1))
    y_val_ohe = ohe.transform(y_val.reshape(-1,1))
    print('End data read...')
    
    '''
    print('Svm training...')
    
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7],
                        'C': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7]}]
    clf = GridSearchCV(
                    SVC(probability=True), tuned_parameters, scoring='accuracy'
                )
    '''
    tuned_parameters = [{'max_depth': [2, 4, 6, 8],
                            'n_estimators': [20, 30, 50, 100, 200],
                         'alpha': [0,0.1,0.2,0.3]}]
    
    
    clf = GridSearchCV(
                    xgb.XGBClassifier(n_jobs=1,use_label_encoder=False,verbosity = 0, random_state=42), tuned_parameters, 
                    scoring='accuracy'
                )
    classes_ = [0,1,2]
    x_train_new, y_train_ohe_new, x_val_early, y_val_early = get_val_set(x_train, y_train_ohe, classes_, percentage = 0.1)
    scaler = StandardScaler()
    x_train_new = scaler.fit_transform(x_train_new)
    eval_set=[(scaler.transform(x_val_early), y_val_early.argmax(axis=1))]
    clf.fit(x_train_new, y_train_ohe_new.argmax(axis=1), early_stopping_rounds=10, eval_set=eval_set, verbose=False)
    print(clf.best_params_)
    best_params = clf.best_params_
    x_train = scaler.fit_transform(x_train)
    train_preds = clf.predict(x_train)
    corrects = np.sum(train_preds == y_train_ohe.argmax(axis=1))
    train_acc = (corrects / x_train.shape[0]) * 100
    train_f1 = f1_score(y_train_ohe.argmax(axis=1), train_preds, average='weighted')
    train_accs.append(train_acc)
    train_F1.append(train_f1)
    train_probs = clf.predict_proba(x_train)
    print('SVM train acc: {}'.format(train_acc))
    print('SVM train F1: {}'.format(train_f1))
    print('CM \n')
    print(confusion_matrix(y_train_ohe.argmax(axis=1), train_preds))
    
    #svm_ = SVC(**best_params)
    #print(clf.best_params_)
    x_val = scaler.transform(x_val)
    test_preds = clf.predict(x_val)
    corrects = np.sum(test_preds == y_val_ohe.argmax(axis=1))
    test_acc = (corrects / x_val.shape[0]) * 100
    test_f1 = f1_score(y_val_ohe.argmax(axis=1), test_preds, average='weighted')
    test_accs.append(test_acc)
    test_F1.append(test_f1)
    test_probs = clf.predict_proba(x_val)
    print('SVM test acc: {}'.format(test_acc))
    print('SVM test F1: {}'.format(test_f1))
    print('CM \n')
    print(confusion_matrix(y_val_ohe.argmax(axis=1), test_preds))
    
    print("Saving SVM predictions... \n")
    
    sheet_name = 'split_'+str(split)
    
    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_val
    data['Preds'] = test_preds
    data['Prob LUAD'] = test_probs[:, 0]
    data['Prob HLT'] = test_probs[:, 1]
    data['Prob LUSC'] = test_probs[:, 2]
    data['Real'] = y_val_ohe.argmax(axis=1)
    data.to_excel(writer_test, sheet_name = sheet_name)

    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_train
    data['Preds'] = train_preds
    data['Prob LUAD'] = train_probs[:, 0]
    data['Prob HLT'] = train_probs[:, 1]
    data['Prob LUSC'] = train_probs[:, 2]
    data['Real'] = y_train_ohe.argmax(axis=1)
    data.to_excel(writer_train, sheet_name=sheet_name)

writer_train.close()
writer_test.close()

In [ ]:
print('Mean Acc in train: {}+-{}'.format(np.mean(train_accs),np.std(train_accs)))
print('Mean F1 in train: {}+-{}'.format(np.mean(train_F1)*100,np.std(train_F1)*100))
print(10*'-')
print('Mean Acc in test: {}+-{}'.format(np.mean(test_accs),np.std(test_accs)))
print('Mean F1 in test: {}+-{}'.format(np.mean(test_F1)*100,np.std(test_F1)*100))

In [ ]:
print('Mean Acc in train: {}+-{}'.format(np.mean(train_accs),np.std(train_accs)))
print('Mean F1 in train: {}+-{}'.format(np.mean(train_F1)*100,np.std(train_F1)*100))
print(10*'-')
print('Mean Acc in test: {}+-{}'.format(np.mean(test_accs),np.std(test_accs)))
print('Mean F1 in test: {}+-{}'.format(np.mean(test_F1)*100,np.std(test_F1)*100))

In [ ]:
from tqdm.notebook import tqdm as tqdm2
# All range of genes results
path = 'Splits_10CV/'


range_genes = 16
all_labels = data_all.iloc[-1,:]
pbar = tqdm2(range(1,range_genes))
global_train_accs = {'mean':[],'std':[]}
global_test_accs = {'mean':[],'std':[]}
global_train_f1 = {'mean':[],'std':[]}
global_test_f1 = {'mean':[],'std':[]}
for n_genes in pbar:
    pbar.set_description('Working with {} genes'.format(n_genes))
    train_accs = []
    test_accs = []
    train_F1 = []
    test_F1 = []
    for i in range(10):
        print(10*'-')
        print('Split {}/{}'.format(i,10))
        print(10*'-')

        print('Data read...')
        data = pd.read_csv('mrmrDEGs/mrmrDEGs_LC_CNV_3classes_p0-001_m0-1_cov3_split'+str(i)+'.csv')

        train_f = open(path+'train_'+str(i)+'.txt', 'r')
        train_caseids = train_f.readlines()
        train_f.close()
        val_f = open(path+'val_'+str(i)+'.txt', 'r')
        val_caseids = val_f.readlines()
        val_f.close()

        train_cids = []
        for cid in train_caseids:
            train_cids.append(cid.replace('\n', ''))

        val_cids = []
        for cid in val_caseids:
            val_cids.append(cid.replace('\n', '')) 

        train_final = []
        for i in range(len(list(data['Case_IDs'].values))):
            resu = re.match('|'.join(train_cids),list(data['Case_IDs'].values)[i])
            if resu:
                if resu.group(0) != '':
                    train_final.append(i)

        val_final = []
        for i in range(len(list(data['Case_IDs'].values))):
            resu = re.match('|'.join(val_cids),list(data['Case_IDs'].values)[i])
            if resu:
                if resu.group(0) != '':
                    val_final.append(i)

        #train_final.insert(0, 1)
        #val_final.insert(0, 1)
        df_train = data.iloc[train_final,]
        df_val = data.iloc[val_final,]

        case_ids_val = df_val['Case_IDs']
        #val_df_all = data_all[case_ids_val]
        y_val = all_labels[case_ids_val].values
        y_val = np.where(y_val == 'Blood Derived Normal', 'Healthy', y_val)
        y_val = np.where(y_val == 'Solid Tissue Normal', 'Healthy', y_val)

        case_ids_train = df_train['Case_IDs']
        #train_df_all = data_all[case_ids_train]
        y_train = all_labels[case_ids_train].values
        y_train = np.where(y_train == 'Blood Derived Normal', 'Healthy', y_train)
        y_train = np.where(y_train == 'Solid Tissue Normal', 'Healthy', y_train)

        x_train = df_train.iloc[:,1:n_genes+1].values
        x_val = df_val.iloc[:,1:n_genes+1].values
        y_train_ohe = ohe.transform(y_train.reshape(-1,1))
        y_val_ohe = ohe.transform(y_val.reshape(-1,1))
        print('End data read...')

        print('Svm training...')
        tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7],
                            'C': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7]}]
        clf = GridSearchCV(
                        SVC(probability=True), tuned_parameters, scoring='accuracy'
                    )

        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        clf.fit(x_train, y_train_ohe.argmax(axis=1))
        print(clf.best_params_)
        best_params = clf.best_params_
        train_preds = clf.predict(x_train)
        corrects = np.sum(train_preds == y_train_ohe.argmax(axis=1))
        train_acc = (corrects / x_train.shape[0]) * 100
        train_f1 = f1_score(y_train_ohe.argmax(axis=1), train_preds, average='weighted', labels=[1, 0, 2])
        train_accs.append(train_acc)
        train_F1.append(train_f1*100)
        print('SVM train acc: {}'.format(train_acc))
        print('SVM train F1: {}'.format(train_f1))
        print('CM \n')
        print(confusion_matrix(y_train_ohe.argmax(axis=1), train_preds, labels=[1, 0, 2]))

        svm_ = SVC(**best_params)
        #print(clf.best_params_)
        x_val = scaler.transform(x_val)
        test_preds = clf.predict(x_val)
        corrects = np.sum(test_preds == y_val_ohe.argmax(axis=1))
        test_acc = (corrects / x_val.shape[0]) * 100
        test_f1 = f1_score(y_val_ohe.argmax(axis=1), test_preds, average='weighted', labels=[1, 0, 2])
        test_accs.append(test_acc)
        test_F1.append(test_f1*100)
        print('SVM test acc: {}'.format(test_acc))
        print('SVM test F1: {}'.format(test_f1))
        print('CM \n')
        print(confusion_matrix(y_val_ohe.argmax(axis=1), test_preds,labels=[1, 0, 2]))
    print('Mean Acc in train: {}+-{}'.format(np.mean(train_accs),np.std(train_accs)))
    print('Mean F1 in train: {}+-{}'.format(np.mean(train_F1),np.std(train_F1)))
    print(10*'-')
    print('Mean Acc in test: {}+-{}'.format(np.mean(test_accs),np.std(test_accs)))
    print('Mean F1 in test: {}+-{}'.format(np.mean(test_F1),np.std(test_F1)))
    
    global_train_accs['mean'].append(np.mean(train_accs))
    global_test_accs['mean'].append(np.mean(test_accs))
    global_train_f1['mean'].append(np.mean(train_F1))
    global_test_f1['mean'].append(np.mean(test_F1))
    global_train_accs['std'].append(np.std(train_accs))
    global_test_accs['std'].append(np.std(test_accs))
    global_train_f1['std'].append(np.std(train_F1))
    global_test_f1['std'].append(np.std(test_F1))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Plot results for range of genes
fig, ax1 = plt.subplots()
plt.title('Accuracy CNV')
color = 'tab:red'
ax1.set_xlabel('#genes')
ax1.set_ylabel('Accuracy')
ax1.plot(list(range(1,16)), global_test_accs['mean'], color="blue")
ax1.tick_params(axis='y')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('std')  # we already handled the x-label with ax1
ax2.plot(list(range(1,16)), global_test_accs['std'], color='red')
ax2.tick_params(axis='y')
ax1.legend()
ax2.legend()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.savefig('plots/acc_range_p0-001_m0-1_cov3.png', format='png', dpi=300)
plt.show()


plt.close()

# Plot results for range of genes
fig, ax1 = plt.subplots()
plt.title('F1 CNV')
color = 'tab:red'
ax1.set_xlabel('#genes')
ax1.set_ylabel('F1-Score')
ax1.plot(list(range(1,16)), global_test_f1['mean'], color="blue")
ax1.tick_params(axis='y')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('std')  # we already handled the x-label with ax1
ax2.plot(list(range(1,16)), global_test_f1['std'], color='red')
ax2.tick_params(axis='y')
ax1.legend()
ax2.legend()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.savefig('plots/f1_range_p0-001_m0-1_cov3.png', format='png', dpi=300)
plt.show()


plt.close()

In [ ]:
x_train = df_train.iloc[:,1:7].values
x_val = df_val.iloc[:,1:7].values
y_train_ohe = ohe.transform(y_train.reshape(-1,1))
y_val_ohe = ohe.transform(y_val.reshape(-1,1))

In [ ]:
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7],
                        'C': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7]}]
clf = GridSearchCV(
                SVC(probability=True), tuned_parameters, scoring='accuracy'
            )

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
clf.fit(x_train, y_train_ohe.argmax(axis=1))
print(clf.best_params_)
best_params = clf.best_params_
train_preds = clf.predict(x_train)
corrects = np.sum(train_preds == y_train_ohe.argmax(axis=1))
train_acc = (corrects / x_train.shape[0]) * 100
print('kNN train acc: {}'.format(train_acc))
print('CM \n')
print(confusion_matrix(y_train_ohe.argmax(axis=1), train_preds, labels=[1, 0, 2]))

In [ ]:
cm = confusion_matrix(y_train_ohe.argmax(axis=1), train_preds, labels=[1, 0, 2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=['LUAD', 'Healthy', 'LUSC'])
disp.plot(cmap= "Blues")

In [ ]:
svm_ = SVC(**best_params)
#print(clf.best_params_)
x_val = scaler.transform(x_val)
test_preds = clf.predict(x_val)
corrects = np.sum(test_preds == y_val_ohe.argmax(axis=1))
test_acc = (corrects / x_val.shape[0]) * 100
print('kNN test acc: {}'.format(test_acc))
print('CM \n')
print(confusion_matrix(y_val_ohe.argmax(axis=1), test_preds,labels=[1, 0, 2]))

In [ ]:
cm = confusion_matrix(y_val_ohe.argmax(axis=1), test_preds, labels=[1, 0, 2])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                             display_labels=['LUAD', 'Healthy', 'LUSC'])
disp.plot(cmap= "Blues")

## TSNE feature visualiztion

In [ ]:
import os
os.mkdir('plots')

In [ ]:
from sklearn.manifold import TSNE
import matplotlib
import matplotlib.pyplot as plt
X_embedded = TSNE(n_components=2).fit_transform(x_val)

hlt = np.where(y_val == 'Healthy')[0]
luad = np.where(y_val == 'LUAD')[0]
lusc = np.where(y_val == 'LUSC')[0]

X_embedded_hlt = X_embedded[hlt,:]
X_embedded_luad = X_embedded[luad,:]
X_embedded_lusc = X_embedded[lusc,:]

plt.figure()
plt.scatter(X_embedded_hlt[:, 0], X_embedded_hlt[:, 1], c='blue', label='HLT')
plt.scatter(X_embedded_luad[:, 0], X_embedded_luad[:, 1], c='green', label='LUAD')
plt.scatter(X_embedded_lusc[:, 0], X_embedded_lusc[:, 1], c='red', label='LUSC')
plt.legend()
plt.savefig('plots/tsne_lastfold_cov2.png', format='png', dpi=300)
plt.show()

## Test with cov == 3

In [ ]:
data = pd.read_csv('gene_matrix_CNV_noNA.csv.gz', compression='gzip')

In [ ]:
path = 'Splits_10CV/'
for split in range(0,10):
    print('{}/{}'.format(split,9))
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 
        
    train_final = []
    for i in range(len(list(data.columns))):
        resu = re.match('|'.join(train_cids),list(data.columns)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for i in range(len(list(data.columns))):
        resu = re.match('|'.join(val_cids),list(data.columns)[i])
        if resu:
            if resu.group(0) != '':
                val_final.append(i)

    train_final.insert(0, 1)
    val_final.insert(0, 1)
    df_train = data.iloc[:,train_final]
    df_val = data.iloc[:,val_final]
    labels_train = df_train.iloc[-1,:].values
    labels_train = np.where(labels_train == 'Blood Derived Normal', 'Healthy', labels_train)
    labels_train = np.where(labels_train == 'Solid Tissue Normal', 'Healthy', labels_train)

    p_values_df = pd.read_csv('p-values-matrix/p_values_train'+str(split)+'.csv')
    decpgs = []
    for _, row in tqdm(p_values_df.iterrows()):
        comp_counter = 0
        if row['HltLuad'] <= 0.001/3 and row['MeanHltLuad'] >= 0.2:
                comp_counter += 1
        if row['HltLusc'] <= 0.001/3 and row['MeanHltLusc'] >= 0.2:
            comp_counter += 1
        if row['LuscLuad'] <= 0.001/3 and row['MeanLuscLuad'] >= 0.2:
            comp_counter += 1
        
        if comp_counter >= 1:
            decpgs.append(row[0])
    
    decpgs.append('X')

    train_degs = df_train.loc[df_train['gene_name'].isin(decpgs)]
    train_degs.to_csv('train_degs2/DEGs_CNV_train'+str(split)+'_p0-001_m0-2_cov1.csv', index=False)

## Test ML with noBDN

In [ ]:
data_all = pd.read_csv('gene_matrix_CNV_noNA.csv.gz', compression='gzip')

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, f1_score

classes = np.array(['healthy', 'adeno', 'squa'])

ohe = preprocessing.OneHotEncoder(sparse=False)
ohe.fit(classes.reshape(-1,1))
ohe.transform(np.array(['adeno']).reshape(-1,1))

In [ ]:
from openpyxl import load_workbook
path = 'Splits_10CV/'
train_accs = []
test_accs = []
train_F1 = []
test_F1 = []

n_genes = 9
writer_test = pd.ExcelWriter('results_excels/CNV_'+str(n_genes)+'gen_noBDN_test.xlsx', engine='openpyxl') 
writer_train = pd.ExcelWriter('results_excels/CNV_'+str(n_genes)+'gen_noBDN_train.xlsx', engine='openpyxl') 
all_labels = data_all.iloc[-1,:]
for split in range(10):
    print(10*'-')
    print('Split {}/{}'.format(split,10))
    print(10*'-')
    
    print('Data read...')
    data = pd.read_csv('mrmrDEGs/mrmrDEGs_LC_CNV_3classes_p0-001_m0-1_cov3_split'+str(split)+'_noBDN.csv')
    
    train_f = open(path+'train_'+str(split)+'.txt', 'r')
    train_caseids = train_f.readlines()
    train_f.close()
    val_f = open(path+'val_'+str(split)+'.txt', 'r')
    val_caseids = val_f.readlines()
    val_f.close()

    train_cids = []
    for cid in train_caseids:
        train_cids.append(cid.replace('\n', ''))

    val_cids = []
    for cid in val_caseids:
        val_cids.append(cid.replace('\n', '')) 

    train_final = []
    for i in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(train_cids),list(data['Case_IDs'].values)[i])
        if resu:
            if resu.group(0) != '':
                train_final.append(i)

    val_final = []
    for j in range(len(list(data['Case_IDs'].values))):
        resu = re.match('|'.join(val_cids),list(data['Case_IDs'].values)[j])
        if resu:
            if resu.group(0) != '':
                val_final.append(j)

    #train_final.insert(0, 1)
    #val_final.insert(0, 1)
    df_train = data.iloc[train_final,]
    df_val = data.iloc[val_final,]
    
    #val_df_all = data_all[case_ids_val]
    case_ids_val = df_val['Case_IDs']
    y_val = all_labels[case_ids_val].values
    no_BDN_val = np.where(y_val != 'Blood Derived Normal')[0]
    no_BDN_val = no_BDN_val.tolist()
    df_val = df_val.iloc[no_BDN_val,:]
    y_val = y_val[no_BDN_val]
    case_ids_val = case_ids_val.iloc[no_BDN_val]

    #y_val = np.where(y_val == 'Blood Derived Normal', 'healthy', y_val)
    y_val = np.where(y_val == 'Solid Tissue Normal', 'healthy', y_val)
    y_val = np.where(y_val == 'LUAD', 'adeno', y_val)
    y_val = np.where(y_val == 'LUSC', 'squa', y_val)
    
    #train_df_all = data_all[case_ids_train]
    case_ids_train = df_train['Case_IDs']
    y_train = all_labels[case_ids_train].values
    no_BDN_train = np.where(y_train != 'Blood Derived Normal')[0]
    no_BDN_train = no_BDN_train.tolist()
    df_train = df_train.iloc[no_BDN_train,:]
    y_train = y_train[no_BDN_train]
    case_ids_train = case_ids_train.iloc[no_BDN_train]
    
    #y_train = np.where(y_train == 'Blood Derived Normal', 'healthy', y_train)
    y_train = np.where(y_train == 'Solid Tissue Normal', 'healthy', y_train)
    y_train = np.where(y_train == 'LUAD', 'adeno', y_train)
    y_train = np.where(y_train == 'LUSC', 'squa', y_train)
    
    x_train = df_train.iloc[:,1:n_genes+1].values
    x_val = df_val.iloc[:,1:n_genes+1].values
    y_train_ohe = ohe.transform(y_train.reshape(-1,1))
    y_val_ohe = ohe.transform(y_val.reshape(-1,1))
    print('End data read...')
    
    print('Svm training...')
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7],
                        'C': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7]}]
    clf = GridSearchCV(
                    SVC(probability=True), tuned_parameters, scoring='accuracy'
                )

    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    clf.fit(x_train, y_train_ohe.argmax(axis=1))
    print(clf.best_params_)
    best_params = clf.best_params_
    train_preds = clf.predict(x_train)
    corrects = np.sum(train_preds == y_train_ohe.argmax(axis=1))
    train_acc = (corrects / x_train.shape[0]) * 100
    train_f1 = f1_score(y_train_ohe.argmax(axis=1), train_preds, average='weighted')
    train_accs.append(train_acc)
    train_F1.append(train_f1)
    train_probs = clf.predict_proba(x_train)
    print('SVM train acc: {}'.format(train_acc))
    print('SVM train F1: {}'.format(train_f1))
    print('CM \n')
    print(confusion_matrix(y_train_ohe.argmax(axis=1), train_preds))
    
    svm_ = SVC(**best_params)
    #print(clf.best_params_)
    x_val = scaler.transform(x_val)
    test_preds = clf.predict(x_val)
    corrects = np.sum(test_preds == y_val_ohe.argmax(axis=1))
    test_acc = (corrects / x_val.shape[0]) * 100
    test_f1 = f1_score(y_val_ohe.argmax(axis=1), test_preds, average='weighted')
    test_accs.append(test_acc)
    test_F1.append(test_f1)
    test_probs = clf.predict_proba(x_val)
    print('SVM test acc: {}'.format(test_acc))
    print('SVM test F1: {}'.format(test_f1))
    print('CM \n')
    print(confusion_matrix(y_val_ohe.argmax(axis=1), test_preds))
    
    print("Saving SVM predictions... \n")
    
    sheet_name = 'split_'+str(split)
    
    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_val
    data['Preds'] = test_preds
    data['Prob LUAD'] = test_probs[:, 0]
    data['Prob HLT'] = test_probs[:, 1]
    data['Prob LUSC'] = test_probs[:, 2]
    data['Real'] = y_val_ohe.argmax(axis=1)
    data.to_excel(writer_test, sheet_name = sheet_name)

    data = pd.DataFrame()
    data['Case_Ids'] = case_ids_train
    data['Preds'] = train_preds
    data['Prob LUAD'] = train_probs[:, 0]
    data['Prob HLT'] = train_probs[:, 1]
    data['Prob LUSC'] = train_probs[:, 2]
    data['Real'] = y_train_ohe.argmax(axis=1)
    data.to_excel(writer_train, sheet_name=sheet_name)

writer_test.close()
writer_train.close()

In [ ]:
print('Mean Acc in train: {}+-{}'.format(np.mean(train_accs),np.std(train_accs)))
print('Mean F1 in train: {}+-{}'.format(np.mean(train_F1)*100,np.std(train_F1)*100))
print(10*'-')
print('Mean Acc in test: {}+-{}'.format(np.mean(test_accs),np.std(test_accs)))
print('Mean F1 in test: {}+-{}'.format(np.mean(test_F1)*100,np.std(test_F1)*100))

## Test better number of genes

In [ ]:
from tqdm.notebook import tqdm as tqdm2
# All range of genes results
path = 'Splits_10CV/'


range_genes = 16
all_labels = data_all.iloc[-1,:]
pbar = tqdm2(range(1,range_genes))
global_train_accs = {'mean':[],'std':[]}
global_test_accs = {'mean':[],'std':[]}
global_train_f1 = {'mean':[],'std':[]}
global_test_f1 = {'mean':[],'std':[]}
for n_genes in pbar:
    pbar.set_description('Working with {} genes'.format(n_genes))
    train_accs = []
    test_accs = []
    train_F1 = []
    test_F1 = []
    for split in range(10):
        print(10*'-')
        print('Split {}/{}'.format(split,10))
        print(10*'-')

        print('Data read...')
        data = pd.read_csv('mrmrDEGs/mrmrDEGs_LC_CNV_3classes_p0-001_m0-1_cov3_split'+str(split)+'_noBDN.csv')

        train_f = open(path+'train_'+str(split)+'.txt', 'r')
        train_caseids = train_f.readlines()
        train_f.close()
        val_f = open(path+'val_'+str(split)+'.txt', 'r')
        val_caseids = val_f.readlines()
        val_f.close()

        train_cids = []
        for cid in train_caseids:
            train_cids.append(cid.replace('\n', ''))

        val_cids = []
        for cid in val_caseids:
            val_cids.append(cid.replace('\n', '')) 

        train_final = []
        for i in range(len(list(data['Case_IDs'].values))):
            resu = re.match('|'.join(train_cids),list(data['Case_IDs'].values)[i])
            if resu:
                if resu.group(0) != '':
                    train_final.append(i)

        val_final = []
        for j in range(len(list(data['Case_IDs'].values))):
            resu = re.match('|'.join(val_cids),list(data['Case_IDs'].values)[j])
            if resu:
                if resu.group(0) != '':
                    val_final.append(j)

        #train_final.insert(0, 1)
        #val_final.insert(0, 1)
        df_train = data.iloc[train_final,]
        df_val = data.iloc[val_final,]

        case_ids_val = df_val['Case_IDs']
        #val_df_all = data_all[case_ids_val]
        y_val = all_labels[case_ids_val].values
        no_BDN_val = np.where(y_val != 'Blood Derived Normal')[0]
        no_BDN_val = no_BDN_val.tolist()
        df_val = df_val.iloc[no_BDN_val,:]
        y_val = y_val[no_BDN_val]

        #y_val = np.where(y_val == 'Blood Derived Normal', 'healthy', y_val)
        y_val = np.where(y_val == 'Solid Tissue Normal', 'healthy', y_val)
        y_val = np.where(y_val == 'LUAD', 'adeno', y_val)
        y_val = np.where(y_val == 'LUSC', 'squa', y_val)

        case_ids_train = df_train['Case_IDs']
        #train_df_all = data_all[case_ids_train]
        y_train = all_labels[case_ids_train].values
        no_BDN_train = np.where(y_train != 'Blood Derived Normal')[0]
        no_BDN_train = no_BDN_train.tolist()
        df_train = df_train.iloc[no_BDN_train,:]
        y_train = y_train[no_BDN_train]
        #y_train = np.where(y_train == 'Blood Derived Normal', 'healthy', y_train)
        y_train = np.where(y_train == 'Solid Tissue Normal', 'healthy', y_train)
        y_train = np.where(y_train == 'LUAD', 'adeno', y_train)
        y_train = np.where(y_train == 'LUSC', 'squa', y_train)

        x_train = df_train.iloc[:,1:n_genes+1].values
        x_val = df_val.iloc[:,1:n_genes+1].values
        y_train_ohe = ohe.transform(y_train.reshape(-1,1))
        y_val_ohe = ohe.transform(y_val.reshape(-1,1))
        print('End data read...')

        print('Svm training...')
        tuned_parameters = [{'kernel': ['rbf'], 'gamma': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7],
                            'C': [2**-7, 2**-5, 2**-2, 2, 2**4, 2**7]}]
        clf = GridSearchCV(
                        SVC(probability=True), tuned_parameters, scoring='accuracy'
                    )

        scaler = StandardScaler()
        x_train = scaler.fit_transform(x_train)
        clf.fit(x_train, y_train_ohe.argmax(axis=1))
        print(clf.best_params_)
        best_params = clf.best_params_
        train_preds = clf.predict(x_train)
        corrects = np.sum(train_preds == y_train_ohe.argmax(axis=1))
        train_acc = (corrects / x_train.shape[0]) * 100
        train_f1 = f1_score(y_train_ohe.argmax(axis=1), train_preds, average='weighted')
        train_accs.append(train_acc)
        train_F1.append(train_f1*100)
        print('SVM train acc: {}'.format(train_acc))
        print('SVM train F1: {}'.format(train_f1))
        print('CM \n')
        print(confusion_matrix(y_train_ohe.argmax(axis=1), train_preds))

        svm_ = SVC(**best_params)
        #print(clf.best_params_)
        x_val = scaler.transform(x_val)
        test_preds = clf.predict(x_val)
        corrects = np.sum(test_preds == y_val_ohe.argmax(axis=1))
        test_acc = (corrects / x_val.shape[0]) * 100
        test_f1 = f1_score(y_val_ohe.argmax(axis=1), test_preds, average='weighted')
        test_accs.append(test_acc)
        test_F1.append(test_f1*100)
        print('SVM test acc: {}'.format(test_acc))
        print('SVM test F1: {}'.format(test_f1))
        print('CM \n')
        print(confusion_matrix(y_val_ohe.argmax(axis=1), test_preds))
    print('Mean Acc in train: {}+-{}'.format(np.mean(train_accs),np.std(train_accs)))
    print('Mean F1 in train: {}+-{}'.format(np.mean(train_F1),np.std(train_F1)))
    print(10*'-')
    print('Mean Acc in test: {}+-{}'.format(np.mean(test_accs),np.std(test_accs)))
    print('Mean F1 in test: {}+-{}'.format(np.mean(test_F1),np.std(test_F1)))
    
    global_train_accs['mean'].append(np.mean(train_accs))
    global_test_accs['mean'].append(np.mean(test_accs))
    global_train_f1['mean'].append(np.mean(train_F1))
    global_test_f1['mean'].append(np.mean(test_F1))
    global_train_accs['std'].append(np.std(train_accs))
    global_test_accs['std'].append(np.std(test_accs))
    global_train_f1['std'].append(np.std(train_F1))
    global_test_f1['std'].append(np.std(test_F1))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Plot results for range of genes
fig, ax1 = plt.subplots()
plt.title('Accuracy CNV NoBDN')
color = 'tab:red'
ax1.set_xlabel('#genes')
ax1.set_ylabel('Accuracy')
ax1.plot(list(range(1,16)), global_test_accs['mean'], color="blue")
ax1.tick_params(axis='y')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('std')  # we already handled the x-label with ax1
ax2.plot(list(range(1,16)), global_test_accs['std'], color='red')
ax2.tick_params(axis='y')
ax1.legend()
ax2.legend()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
#plt.savefig('plots/acc_range_p0-001_m0-1_cov3.png', format='png', dpi=300)
plt.show()


plt.close()

# Plot results for range of genes
fig, ax1 = plt.subplots()
plt.title('F1 CNV NoBDN')
color = 'tab:red'
ax1.set_xlabel('#genes')
ax1.set_ylabel('F1-Score')
ax1.plot(list(range(1,16)), global_test_f1['mean'], color="blue")
ax1.tick_params(axis='y')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('std')  # we already handled the x-label with ax1
ax2.plot(list(range(1,16)), global_test_f1['std'], color='red')
ax2.tick_params(axis='y')
ax1.legend()
ax2.legend()
fig.tight_layout()  # otherwise the right y-label is slightly clipped
#plt.savefig('plots/f1_range_p0-001_m0-1_cov3.png', format='png', dpi=300)
plt.show()


plt.close()